In [20]:
import pandas as pd
import numpy as np
kg = pd.read_csv('kg(3in1).csv')
pkg = pd.read_csv('pkg(remove_back_edge).csv')

C:\Users\Tianqi\AppData\Local\Temp\ipykernel_3768\2809246610.py:3: DtypeWarning: Columns (1,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  kg = pd.read_csv('kg(3in1).csv')
C:\Users\Tianqi\AppData\Local\Temp\ipykernel_3768\2809246610.py:4: DtypeWarning: Columns (6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  pkg = pd.read_csv('pkg(remove_back_edge).csv')


In [2]:
kg

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
0,subordinate1,Access to Health Services,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subordinate1,Access to Primary Care,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subordinate1,Health Literacy,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,subordinate1,Health Insurance,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subordinate1,Early Childhood Development and Education,Education access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6664545,ethnicity_disease,African American,premature aging,ethnicity,disease,50.0,NaN,NaN,NaN,C0231341,NaN,D019588
6664546,ethnicity_disease,Black,hepatitis C co-infection,ethnicity,disease,100.0,NaN,NaN,NaN,C0275524,NaN,D060085
6664547,ethnicity_disease,Black,hepatitis C,ethnicity,disease,50.0,NaN,NaN,NaN,C0524910,NaN,D019698
6664548,ethnicity_disease,non-Hispanic Black,knee pain,ethnicity,disease,50.0,NaN,NaN,NaN,C0877149,NaN,D046788


In [12]:
pkg

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id,y_id
0,PHYHIP,gene_gene,KIF15,NaN,gene,gene,NaN,NaN,9796,56992
1,GPANK1,gene_gene,PNMA1,NaN,gene,gene,NaN,NaN,7918,9240
2,ZRSR2,gene_gene,TTC33,NaN,gene,gene,NaN,NaN,8233,23548
3,NRF1,gene_gene,MAN1B1,NaN,gene,gene,NaN,NaN,4899,11253
4,PI4KA,gene_gene,RGS20,NaN,gene,gene,NaN,NaN,5297,8601
...,...,...,...,...,...,...,...,...,...,...
5847647,digestive tract junction,anatomy_anatomy,intestinal junction,NaN,anatomy,anatomy,NaN,NaN,1100000,8410024
5847648,zone of organ,anatomy_anatomy,transition zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410025
5847649,zone of organ,anatomy_anatomy,peripheral zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410026
5847650,zone of organ,anatomy_anatomy,central zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410027


In [21]:
t1 = kg[['head', 'x_id', 'head_type']]
t2 = kg[['tail', 'y_id', 'tail_type']]

In [ ]:
t1 = pkg[['head', 'x_id', 'head_type']]
t2 = pkg[['tail', 'y_id', 'tail_type']]

In [22]:
t2.columns = ['head', 'x_id', 'head_type']
# concat pkg1 and pkg2
testid = pd.concat([t1, t2], axis=0)
testid = testid.drop_duplicates(subset=('head', 'head_type'))
testid['x_index'] = testid.groupby('head_type').cumcount().astype(int)
testid.reset_index(drop=True, inplace=True)
testid

,head,x_id,head_type,x_index
0,Access to Health Services,NaN,subtype,0
1,Access to Primary Care,NaN,subtype,1
2,Health Literacy,NaN,subtype,2
3,Health Insurance,NaN,subtype,3
4,Early Childhood Development and Education,NaN,subtype,4
...,...,...,...,...
139321,estradiol,D004958,drug,8677
139322,reduced cortical,D001523,disease,19671
139323,premature aging,D019588,disease,19672
139324,hepatitis C co-infection,D060085,disease,19673


In [23]:
# get unique 'head_type'
h = testid['head_type'].unique()
len(h),h

(22,
 array(['subtype', 'SDoH', 'gene', 'drug', 'effect/phenotype', 'disease',
        'biological_process', 'molecular_function', 'cellular_component',
        'exposure', 'pathway', 'anatomy', 'age', 'ethnicity', 'religion',
        'marital_status', 'gender', 'insurance', 'species', 'variant',
        'cellline', 'type'], dtype=object))

In [24]:
id_type = testid['head_type'].unique() 
dfs_by_type = {x_type: df_sub for x_type, df_sub in testid.groupby('head_type')}

for i in range(len(id_type)):
    subdf = dfs_by_type.get(id_type[i])
    subdf = subdf[['x_index']]
    subdf.columns = ['node_id']
    path = r'C:\data\RA\shenlab\SDoH&KG\extension\data\kg\nodes_'+str(i)+ '.csv'
    subdf.to_csv(path, index=False)


In [25]:
testdf = kg.merge(testid[['head', 'head_type', 'x_index']], how='left', left_on=['head', 'head_type'], right_on=['head', 'head_type'])
#testdf = pkg.merge(testid[['head', 'head_type', 'x_index']], how='left', left_on=['head', 'head_type'], right_on=['head', 'head_type'])
# testdf = testdf.drop(columns=['x_index_x'])
# testdf.rename(columns={'x_index_y': 'x_index'}, inplace=True)

# # Merge to replace y_index
testdf = testdf.merge(testid[['head', 'head_type', 'x_index']], how='left', left_on=['tail', 'tail_type'], right_on=['head', 'head_type'])
testdf = testdf.drop(columns=['head_y', 'head_type_y'])
testdf.rename(columns={'x_index_y': 'y_index'}, inplace=True)
testdf.rename(columns={'x_index_x': 'x_index'}, inplace=True)
testdf.rename(columns={'head_x': 'head'}, inplace=True)
testdf.rename(columns={'head_type_x': 'head_type'}, inplace=True)

testdf 

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id,x_index,y_index
0,subordinate1,Access to Health Services,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,subordinate1,Access to Primary Care,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,subordinate1,Health Literacy,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
3,subordinate1,Health Insurance,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
4,subordinate1,Early Childhood Development and Education,Education access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6664545,ethnicity_disease,African American,premature aging,ethnicity,disease,50.0,NaN,NaN,NaN,C0231341,NaN,D019588,23,19672
6664546,ethnicity_disease,Black,hepatitis C co-infection,ethnicity,disease,100.0,NaN,NaN,NaN,C0275524,NaN,D060085,22,19673
6664547,ethnicity_disease,Black,hepatitis C,ethnicity,disease,50.0,NaN,NaN,NaN,C0524910,NaN,D019698,22,18899
6664548,ethnicity_disease,non-Hispanic Black,knee pain,ethnicity,disease,50.0,NaN,NaN,NaN,C0877149,NaN,D046788,43,19674


In [26]:
t3 = testdf[['x_index','head_type','relation', 'y_index','tail_type']] 
t3 = t3.dropna(subset=['x_index', 'y_index'])
t3['x_index'] = t3['x_index'].astype(int)
t3['y_index'] = t3['y_index'].astype(int)
t3

,x_index,head_type,relation,y_index,tail_type
0,0,subtype,subordinate1,0,type
1,1,subtype,subordinate1,0,type
2,2,subtype,subordinate1,0,type
3,3,subtype,subordinate1,0,type
4,4,subtype,subordinate1,1,type
...,...,...,...,...,...
6664545,23,ethnicity,ethnicity_disease,19672,disease
6664546,22,ethnicity,ethnicity_disease,19673,disease
6664547,22,ethnicity,ethnicity_disease,18899,disease
6664548,43,ethnicity,ethnicity_disease,19674,disease


In [27]:
mapping_dict = {value: index for index, value in enumerate(id_type)}
mapping_dict

{'subtype': 0,
 'SDoH': 1,
 'gene': 2,
 'drug': 3,
 'effect/phenotype': 4,
 'disease': 5,
 'biological_process': 6,
 'molecular_function': 7,
 'cellular_component': 8,
 'exposure': 9,
 'pathway': 10,
 'anatomy': 11,
 'age': 12,
 'ethnicity': 13,
 'religion': 14,
 'marital_status': 15,
 'gender': 16,
 'insurance': 17,
 'species': 18,
 'variant': 19,
 'cellline': 20,
 'type': 21}

In [28]:
grouped = t3.groupby(['head_type', 'tail_type'])

# Create a dictionary to store the sub-DataFrames
subdfs = {group: subdf for group, subdf in grouped}
for key, subdf in subdfs.items():
    print(f"Sub-DataFrame for head_type = {key[0]} and tail_type = {key[1]}:")
    print(subdf)
    print("\n")
    df_out = subdf[['x_index', 'y_index']]
    # rename the columns
    df_out.columns = ['src_id', 'dst_id']
    # save the sub-DataFrame to a CSV file
    df_out.to_csv(f'C:/data/RA/shenlab/SDoH&KG/extension/data/kg/edges_{mapping_dict[key[0]]}_{mapping_dict[key[1]]}.csv', index=False)
    #df_out.to_csv(f'C:/data/RA/shenlab/SDoH&KG/extension/data/pkg/edges_{mapping_dict[key[0]]}_{mapping_dict[key[1]]}.csv', index=False)

Sub-DataFrame for head_type = SDoH and tail_type = subtype:
      x_index head_type      relation  y_index tail_type
27          0      SDoH  subordinate0        4   subtype
28          1      SDoH  subordinate0       22   subtype
29          2      SDoH  subordinate0       14   subtype
30          3      SDoH  subordinate0       19   subtype
31          4      SDoH  subordinate0       22   subtype
...       ...       ...           ...      ...       ...
6544     6483      SDoH  subordinate0       23   subtype
6545     6484      SDoH  subordinate0       11   subtype
6546     6485      SDoH  subordinate0       16   subtype
6547     6486      SDoH  subordinate0       16   subtype
6548     6487      SDoH  subordinate0       20   subtype

[6522 rows x 5 columns]


Sub-DataFrame for head_type = age and tail_type = disease:
         x_index head_type     relation  y_index tail_type
6658224        0       age  age_disease    18938   disease
6658225        0       age  age_disease    18939   d

In [12]:
testid

,head,x_id,head_type,x_index
0,Access to Health Services,NaN,subtype,0
1,Access to Primary Care,NaN,subtype,1
2,Health Literacy,NaN,subtype,2
3,Health Insurance,NaN,subtype,3
4,Early Childhood Development and Education,NaN,subtype,4
...,...,...,...,...
138775,estradiol,D004958,drug,8629
138776,reduced cortical,D001523,disease,19172
138777,premature aging,D019588,disease,19173
138778,hepatitis C co-infection,D060085,disease,19174


In [22]:
testid

,head,x_id,head_type,x_index
0,PHYHIP,9796,gene,0
1,GPANK1,7918,gene,1
2,ZRSR2,8233,gene,2
3,NRF1,4899,gene,3
4,PI4KA,5297,gene,4
...,...,...,...,...
129352,SLC44A3-AS1,101928079,gene,27668
129353,ZNRD2-DT,254100,gene,27669
129354,MIR548F3,100302159,gene,27670
129355,multi-cellular organism,468,anatomy,14033


In [19]:
testid.to_csv('testid_pkg.csv', index=False)

In [30]:
# save testid to csv
testid.to_csv('testid_kg.csv', index=False)

: 

In [12]:
import pickle
with open('C:\data\RA\shenlab\SDoH&KG\extension\potential_3ca8_gene_SDoH.pkl', 'rb') as f:
    data = pickle.load(f)

data

(array([15721, 13171, 15721, ..., 23447, 18189, 10968], dtype=int64),
 array([5572, 5894, 5510, ...,  758, 6015, 5403], dtype=int64))

In [13]:
# get the dataframe where head_type is gene
gene_id = testid[testid['head_type']=='gene']
SDoh_id = testid[testid['head_type']=='SDoH']

In [19]:
src_name = [gene_id['head'].tolist()[i] for i in data[0]]
dst_name = [SDoh_id['head'].tolist()[i] for i in data[1]]

In [22]:
# save the potential edges to a csv file
potential_edges = pd.DataFrame({'head': src_name, 'tail': dst_name})
potential_edges.to_csv('C:\data\RA\shenlab\SDoH&KG\extension\potential_edges_gene_sdoh.csv', index=False)

In [32]:
l = ['clinical practice', 'care-partner outcomes', 'neurologists', 'memory clinic',
        'primary care', 'memory clinical practice', 'diagnostic meeting', 'screening', 'memory clinic setting',
        'clinic', 'therapy','brain MRI', 'clinical settings']

In [33]:
# remove row if potential_edges['tail'] is in l
potential_edges1 = potential_edges[~potential_edges['tail'].isin(l)]
potential_edges1

,head,tail
2,TPK1,residence type
4,ISCA2,degree
5,ISCA2,relationship
9,ISCA2,vaccination
10,DACT3,self-identified race
...,...,...
17988,NPIPB13,environment
17989,TRPC4,community
17990,TRIM33,environmental factors
17993,RALY,residence type


In [36]:
potential_edges1.to_csv('C:\data\RA\shenlab\SDoH&KG\extension\potential_edges_gene_sdoh.csv', index=False)

In [41]:
unique_src = potential_edges1['head'].unique()   
unique_dst = potential_edges1['tail'].unique()
len(unique_src), len(unique_dst)

(758, 34)

In [42]:
unique_dst

array(['residence type', 'degree', 'relationship', 'vaccination',
       'self-identified race', 'expensive', 'examination',
       'nonhouse residence', 'deleterious exposures', 'learning',
       'educational attainment', 'educational levels', 'wealth',
       'utilization charges', 'familial', 'Empathy',
       'Environmental factors', 'pesticide-induced', "family member's",
       'families', 'community settings', 'development',
       'environmental factors', 'region', 'collaboration model',
       'cognitive abilities', 'occupation', 'community', 'grade',
       'family', 'environments', 'educationalfactors', 'education',
       'environment'], dtype=object)

In [43]:
# group the potential edges by tail, put the head to list
grouped = potential_edges1.groupby('tail')
grouped = grouped['head'].apply(list)
grouped  

tail
Empathy                   [DACT3, COQ8B, RAPGEF4-AS1, NPIPB2, ARNTL2-AS1...
Environmental factors     [STK39, COQ8B, GBP6, RPL7AP64, MIR648, RAPGEF4...
cognitive abilities       [TPK1, COQ8B, ATPSCKMT, CEP112, L3MBTL4-AS1, D...
collaboration model       [TPK1, ATPSCKMT, CDH26, SYT10, DACT3, FAM117B,...
community                 [RNF17, ACOT9, MUC13, ARMCX2, OR10G2, FBXL7, N...
community settings        [MIR5189, TPK1, SNORD1B, DACT3, LRRC45, CDCA2,...
degree                    [ISCA2, ARMCX2, OR10G2, ADAP1, MAPK8, FEM1AP1,...
deleterious exposures     [COQ8B, DACT3, ENO4, GET1, ABCC1, RPL7AP64, H2...
development               [TPK1, SNORD1B, DACT3, GBP6, ISCA2, COQ8B, SYT...
education                 [SNORD1B, DACT3, TPK1, ISCA2, C22orf39, COQ8B,...
educational attainment    [ENO4, COQ8B, DACT3, GRID1-AS1, ST8SIA5, GBP6,...
educational levels        [MIR648, STK39, SCA30, GBP6, CEP112, COQ8B, GB...
educationalfactors        [SLC7A4, STK39, SCA30, COQ8B, CEP112, GBP6, KL...
environ

In [44]:
# save the grouped potential edges to a csv file
grouped.to_csv('C:\data\RA\shenlab\SDoH&KG\extension\potential_edges_gene_sdoh_grouped.csv', index=True)


In [45]:
with open('C:\data\RA\shenlab\SDoH&KG\extension\potential_a0d8_gene_gene.pkl', 'rb') as f:
    data1 = pickle.load(f)

data1

([8993,
  16964,
  8993,
  8993,
  3,
  3,
  3,
  3,
  7,
  7,
  7,
  7,
  7,
  6945,
  0,
  11864,
  12,
  12,
  12,
  13825,
  8057,
  8993,
  5593,
  4172,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  24,
  24,
  24,
  24,
  24,
  24,
  24,
  24,
  24,
  9,
  9,
  9,
  9,
  9,
  9,
  721,
  0,
  0,
  0,
  0,
  0,
  24,
  24,
  24,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  9,
  9,
  9,
  8993,
  8993,
  4335,
  24,
  24,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  16964,
  15030,
  15030,
  15030,
  22,
  9698,
  4335,
  9,
  9,
  20,
  15030,
  10920,
  8373,
  10920,
  11864,
  4172,
  6,
  6,
  6,
  6,
  9,
  24,
  4172,
  16964,
  1683,
  8477,
  6,
  6,
  6,
  6,
  6,
  6,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  24,
  15030,
  4172,
  14077,
  10920,
  1683,
  20,
  20,
  7899,
  7899,
  13825,
  18,
  18,
  18,
  18,
  20,
  18,
  18,
  18,
  18,
  13825,
  6,
  6,
  6,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  20,
  20,
  4172,
  16964,
  18,
  18,
  18,
 

In [14]:
gene_id

,head,x_id,head_type,x_index
8818,APOE,348,gene,0
8841,QPCTL,54814,gene,1
8844,APP,351,gene,2
8851,MAPT,4137,gene,3
8897,BACE1,23821,gene,4
...,...,...,...,...
138664,PANDAR,101154753,gene,27670
138665,TP53COR1,102800311,gene,27671
138666,SLC44A3-AS1,101928079,gene,27672
138667,ZNRD2-DT,254100,gene,27673


In [46]:
src_name1 = [gene_id['head'].tolist()[i] for i in data1[0]]
dst_name1 = [gene_id['head'].tolist()[i] for i in data1[1]]
# save the potential edges to a csv file
potential_edges1 = pd.DataFrame({'head': src_name1, 'tail': dst_name1})
potential_edges1.to_csv('C:\data\RA\shenlab\SDoH&KG\extension\potential_edges_gene_gene.csv', index=False)

In [48]:
pkg = pd.read_csv('pkg(remove_back_edge).csv')
pkg

C:\Users\Tianqi\AppData\Local\Temp\ipykernel_7476\1760020873.py:1: DtypeWarning: Columns (6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  pkg = pd.read_csv('pkg(remove_back_edge).csv')


,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id,y_id
0,PHYHIP,gene_gene,KIF15,NaN,gene,gene,NaN,NaN,9796,56992
1,GPANK1,gene_gene,PNMA1,NaN,gene,gene,NaN,NaN,7918,9240
2,ZRSR2,gene_gene,TTC33,NaN,gene,gene,NaN,NaN,8233,23548
3,NRF1,gene_gene,MAN1B1,NaN,gene,gene,NaN,NaN,4899,11253
4,PI4KA,gene_gene,RGS20,NaN,gene,gene,NaN,NaN,5297,8601
...,...,...,...,...,...,...,...,...,...,...
5848019,digestive tract junction,anatomy_anatomy,intestinal junction,NaN,anatomy,anatomy,NaN,NaN,1100000,8410024
5848020,zone of organ,anatomy_anatomy,transition zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410025
5848021,zone of organ,anatomy_anatomy,peripheral zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410026
5848022,zone of organ,anatomy_anatomy,central zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410027


In [51]:
# get list of gene if head_type is gene
gene_id1 = pkg[pkg['head_type']=='gene']
gene_id1 = gene_id1['head'].tolist()
gene_id2 = pkg[pkg['tail_type']=='gene']
gene_id2 = gene_id2['tail'].tolist()
gene_id = gene_id1 + gene_id2
# delete duplicate gene
gene_id = list(set(gene_id))

In [52]:
potential_edges1

,head,tail
0,CYP3A4,ADRA1D
1,ZG16,PSMD4
2,CYP3A4,MAOA
3,CYP3A4,ADRA1A
4,tau,APC
...,...,...
12759,TRAV3,Mr. C
12760,TRAV26-1,Mr. C
12761,TRAV36DV7,Mr. C
12762,TRAV1-1,Mr. C


In [53]:
# if head or tail is not in gene_id, remove the row
potential_edges2 = potential_edges1[potential_edges1['head'].isin(gene_id) & potential_edges1['tail'].isin(gene_id)]
potential_edges2

,head,tail
0,CYP3A4,ADRA1D
1,ZG16,PSMD4
2,CYP3A4,MAOA
3,CYP3A4,ADRA1A
13,IFNA2,NPPB
...,...,...
12515,FOXP3,ROCK1P1
12516,KLHL38,ROCK1P1
12521,SCNM1,ROCK1P1
12700,MDFI,ROCK1P1


In [54]:
# save the potential edges to a csv file
potential_edges2.to_csv('C:\data\RA\shenlab\SDoH&KG\extension\potential_edges_gene_gene.csv', index=False)

: 